In [ ]:
import json, math, random

class Letters:
    def __init__(self, scene, font):
        self.scene = scene
        self.font = font
        
        scene['programs']['letters'] = {
            'vertex': '''
                precision highp float;

                uniform mat4 camera;
                uniform vec2 screen;
                uniform vec3 random;

                attribute vec3 rel_pos;
                attribute vec4 base_pos;
                attribute vec4 crop_info;
                attribute vec4 fill;
                attribute vec4 stroke;

                varying vec4 ci;
                varying vec4 v_fill;
                varying vec4 v_stroke;

                void main(void) {
                  vec4 base_position = camera * vec4(base_pos.xyz, 1.0);
                  vec3 rel_position = vec3(rel_pos.xy / screen, 0.0);
                  float c = base_pos.w / base_position.w;
                  if (c > 2.0) {
                      c = 0.0;
                  }
                  vec4 antialias = vec4((random.xy * 3.0 - 1.5) / screen, 0.0, 0.0);
                  gl_Position = (vec4(base_position.xyz / base_position.w + rel_position * c * 2.0, 1.0) + antialias) * base_position.w;
                  gl_PointSize = rel_pos.z * c;
                  ci = crop_info;
                  v_fill = fill;
                  v_stroke = stroke;
                }
            ''',
            'fragment': '''
                precision highp float;

                uniform sampler2D font;
                uniform vec3 random;

                varying vec4 ci;
                varying vec4 v_fill;
                varying vec4 v_stroke;

                highp float rand(vec2 co)
                {

                  highp float a = 12.9898;
                  highp float b = 78.233;
                  highp float c = 43758.5453;
                  highp float dt = dot(co.xy, vec2(a,b));
                  highp float sn = mod(dt, 3.14);
                  return fract(sin(sn) * c);
                }

                void main(void) {
                  //if (rand(gl_PointCoord.xy + random.xy) < 0.5) discard;
                  vec2 pc = gl_PointCoord;
                  if (pc.x > ci[3] || pc.y * ci[3] > 1.0) discard;
                  vec2 uv = vec2(ci.x, ci.y) + vec2(pc.x * 4.0, pc.y) * ci[2];
                  vec4 col = texture2D(font, uv);
                  if (col.x < 0.5) discard;
                  vec4 res;
                  if (col.y < 0.5) {
                      res = v_stroke;
                      // gl_FragColor = vec4(res.xyz, 1.0);
                      // return;
                  } else {
                      res = v_fill;
                  }
                  float alpha;
                  if ({{do_post}}) {
                      alpha = gl_FragCoord.w;
                  } else {
                      alpha = 1.0;
                  }
                  gl_FragColor = vec4(res.xyz, alpha);
                }
            ''',
            'constants': {
                '{{do_post}}': 'true',
            },
            'attributes': {
                'rel_pos': '3f',
                'base_pos': '4f',
                'fill': '4f',
                'stroke': '4f',
            },
            'uniforms': {
                'camera': 'Matrix4f',
                'screen': '2f',
                'random': '3f',
            },
            'textures': {
                'font': 0,
            },
            'mode': 'POINTS',
        }

        self.letters_rel_pos = []
        scene['buffers']['letters_rel_pos'] = {
             'type': 'Float32Array',
             'data': self.letters_rel_pos,
             'is_index': False,
             'item_size': 3,
        }
        
        self.letters_base_pos = []
        scene['buffers']['letters_base_pos'] = {
             'type': 'Float32Array',
             'data': self.letters_base_pos,
             'is_index': False,
             'item_size': 4,
        }
        
        self.letters_crop_info = []
        scene['buffers']['letters_crop_info'] ={
            'type': 'Float32Array',
            'data': self.letters_crop_info,
            'is_index': False,
            'item_size': 4,
        }
        
        self.letters_fill = []
        scene['buffers']['letters_fill'] ={
            'type': 'Float32Array',
            'data': self.letters_fill,
            'is_index': False,
            'item_size': 4,
        }

        self.letters_stroke = []
        scene['buffers']['letters_stroke'] ={
            'type': 'Float32Array',
            'data': self.letters_stroke,
            'is_index': False,
            'item_size': 4,
        }       
        
        scene['textures'][font['name']] = {
            'filter': 'LINEAR',
            'url': font['image'],
        }
        
        self.draw = {
            'first': 0,
            'count': 0,
        }
        scene['layers']['letters'] = {
            'name': 'letters',
            'program': 'letters',
            'frames': list(range(scene['stop'])),
            'uniforms': {
                'camera': None,
                'screen': None,
                'random': None,
            },
            'attributes': {
                'rel_pos': 'letters_rel_pos',
                'base_pos': 'letters_base_pos',
                'crop_info': 'letters_crop_info',
                'fill': 'letters_fill',
                'stroke': 'letters_stroke',
            },
            'textures': {
                'font': font['name'],
            },
            'draw': self.draw,
        }
        
    def crop_info(self, ch):
        width, height = self.font['width'], self.font['height']
        info = self.font[ch]
        x0 = info['tex_x'] / width
        y0 = info['tex_y'] / height
        ratio = info['tex_w'] / info['tex_h']
        scale = info['tex_h'] / height if ratio < 1 else info['tex_w'] / height
        return x0, y0, scale, ratio

    def type_text(self, text):
        res = []
        x, y = 0, 0
        min_y, max_y = 0, 0
        for ch in text:
            info = self.font[ch]
            min_y = min(min_y, info['y_bearing'])
            max_y = max(max_y, info['y_bearing'] + info['height'])
            max_wh = max(info['width'], info['height'])
            res.append((
                x + info['x_bearing'] * 1 + max_wh / 2,
                y - info['y_bearing'] * 1 - max_wh / 2,
                max(info['tex_w'], info['tex_h']),
            ))
            x += info['x_advance']
            y += info['y_advance']
        x_cnt = x / 2
        y_cnt = (min_y + max_y) / 2
        return [e for (x, y, s) in res for e in (x - x_cnt, y - y_cnt, s)]

    def add_label(self, text, position, size, color):
        self.letters_rel_pos.extend(self.type_text(text))
        for ch in text:
            self.letters_base_pos.extend(position + [size])
            self.letters_crop_info.extend(self.crop_info(ch))
            self.letters_fill.extend(color)
            self.letters_stroke.extend([1, 1, 1, 1])
            self.draw['count'] += 1

font = json.load(open('cmu_serif.json'))

def mix(values0, values1, t):
    return [val0 * (1 - t) + val1 * t for val0, val1 in zip(values0, values1)]

def anim_values(values0, values1, frames):
    res = []
    for frame in range(frames):
        t = frame / (frames - 1)
        res.extend(mix(values0, values1, t))
    return res

frames = 1200

scene = {
    'width': 1366,
    'height': 768,
    'start': 0,
    'step': 1,
    'stop': 10,
    'background': [1, 1, 1, 0],
    'programs': {
        'post': {
            'uniforms': {
                'screen': '2f',
                'random': '2f',
                'focus': '2f',
            },
            'textures': {
                'render': 7,
            },
            'attributes': {
                'pos': '2f',
            },
            'vertex': '''
                precision highp float;
                attribute vec2 pos;
                void main(void) {
                    gl_Position = vec4(pos.xy, 0.0, 1.0);
                }
            ''',
            'fragment': '''
                precision highp float;
                uniform sampler2D render;
                uniform vec2 screen;
                uniform vec2 random;
                uniform vec2 focus;

                highp float rand(vec2 co)
                {

                  highp float a = 12.9898;
                  highp float b = 78.233;
                  highp float c = 43758.5453;
                  highp float dt = dot(co.xy, vec2(a,b));
                  highp float sn = mod(dt, 3.14);
                  return fract(sin(sn) * c);
                }

                void main(void) {
                    vec4 cur_tex = texture2D(render, gl_FragCoord.xy / screen);
                    vec3 sum = cur_tex.xyz;
                    float count = 1.0;
                    float alpha = focus.x;
                    float beta = focus.y;
                    float rad = pow(rand(random + gl_FragCoord.xy), 0.5) * alpha * max(0.0, 1.0 - beta * cur_tex.a);
                    float min_a = (1.0 - rad / alpha) / beta;
                    for (int i=0; i<40; i++) {
                        float a = float(i) + random.x;
                        vec4 tex = texture2D(render, (gl_FragCoord.xy + rad * vec2(cos(a), sin(a))) / screen);
                        if (tex.a > min_a) continue;
                        sum += tex.xyz;
                        count += 1.0;
                    }
                    gl_FragColor = vec4(sum / count, 1.0);
                }
            ''',
            'mode': 'TRIANGLES',
        },
    },
    'buffers': {
        'fullscreen': {
            'type': 'Float32Array',
            'data': [-1, -1, -1, 3, 3, -1],
            'is_index': False,
            'item_size': 2,
        },
    },
    'layers': {},
    'textures': {},
    'camera': {
        'near': 0.1,
        'far': 1000,
        'fov': 45,
        'target': [0, 0, 0],
        'polar': anim_values([0, 0, 1], [180, 180, 3], frames // 2) + anim_values([180, 180, 3], [360, 360, 1], frames // 2),
    },
    'postprocessing': {
        'post_iters': 7,
        'blend_iters': 7,
        'motion_blur': 0,
        'program': 'post',
        'uniforms': {
            'screen': None,
            'random': None,
            'focus': [5, 5],
        },
        'textures': {
            'render': None,
        },
        'attributes': {
            'pos': 'fullscreen',
        },
        'draw': {
            'first': 0,
            'count': 3,
        }
    }
}

sd = json.load(open('../renderer/scenes/scene1.data.json'))

letters = Letters(scene, font)
for i in range(sd['count']):
    letters.add_label(sd['text'][i], sd['pos'][3*i:3*(i+1)], sd['size'][i], sd['color'][4*i:4*(i+1)])

json.dump(scene, open('../renderer/scenes/scene1.base.json', 'w'))

In [ ]:
import math
def smooth(t):
    return (1 - math.cos(t * math.pi)) / 2

def word_pos(word):
    idx = sd['text'].index(word) * 3
    return sd['pos'][idx:idx+3]

shutter = 5

frames = 1
targets = word_pos('reference implementation')
polars = [0, -30, 1]
focuses = [shutter, 4]

def trans(duration, final_target, final_polar, final_focus=None):
    global frames
    duration *= 2
    init_target = targets[-3:]
    init_polar = polars[-3:]
    init_focus = focuses[-2:]
    if type(final_target) is str:
        final_target = word_pos(final_target)
    if final_focus is None:
        final_focus = init_focus
    frames += duration
    for i in range(duration):
        t = smooth(i / (duration - 1))
        targets.extend(mix(init_target, final_target, t))
        polars.extend(mix(init_polar, final_polar, t))
        focuses.extend(mix(init_focus, final_focus, t))
        
# trans(2, 'reference implementation', [0, -30, 1], [1, 1])
trans(30, 'reference implementation', [0, -30, 4], [shutter, 4])
trans(30, 'virtual reality system', [0, -70, 2], [shutter, 10])
trans(70, 'robot learning', [0, -70, 4], [shutter, 10])
trans(30, 'planning task', [20, -30, 4], [shutter, 10])
trans(30, 'domain expertise', [10, -30, 2], [shutter, 10])
trans(30, 'common knowledge', [40, -30, 1], [shutter, 10])
trans(30, 'electrical power', [0, 0, 3], [shutter, 10])
trans(30, 'battery pack', [0, 20, 3], [shutter, 10])
trans(30, 'function type', [50, 20, 3], [shutter, 10])
trans(300, 'function type', [60, 50, 100], [shutter, 30])
trans(200, 'human immunodeficiency virus', [20, -30, 2], [shutter, 10])
trans(100, 'reference implementation', [0, -30, 1], [shutter, 4])
        
do_post = True
json.dump({
    'width': 1920,
    'height': 1080,
    'start': 500,
    'step': 1,
    'export_as': 'first',
    'stop': frames,
    'programs': {
        'letters': {
            'constants': {
                '{{do_post}}': do_post,
            }
        }
    },
    'layers': {
        'letters': {
            'frames': list(range(frames)),
        }
    },
    'camera': {
        'target': targets,
        'polar': polars,
    },
    'postprocessing': {
        'enabled': do_post,
        'motion_blur': 0.3,
        'post_iters': 7,
        'blend_iters': 7,
        'uniforms': {
            'focus': focuses,
        },
    },
}, open('../renderer/scenes/scene1.1.json', 'w'))

In [236]:
frames

1821

In [233]:
frames * 5 / 60 / 60

2.529166666666667

In [207]:
!ffmpeg -i "../renderer/frames/test/%06d.png" -c:v libx264 -preset slow -profile:v high -crf 18 -coder 1 -pix_fmt yuv420p -movflags +faststart -g 30 -bf 2 -c:a aac -b:a 384k -profile:a aac_low "../renderer/videos/test1.mp4"

ffmpeg version 4.0 Copyright (c) 2000-2018 the FFmpeg developers
  built with gcc 8.1.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-avresample --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libass --enable-libbluray --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxml2 --enable-libxvid --enable-nvenc --enable-omx --enable-shared --enable-version3
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    